In [ ]:
!pip install ncps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 3.0 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade tensorflow

In [ ]:
!pip install --upgrade ncps

In [ ]:
!pip install tensorflow==2.12.0

INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.0/586.0 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 MB 10.0 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Unins

In [ ]:
!pip install --upgrade ncps

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from ncps.wirings import AutoNCP
from ncps.tf import LTCCell

# Load dataset
df = pd.read_csv('delhi_aqi.csv')  # Replace with the actual file name

# Convert 'date' column to datetime and extract useful features
df['Datetime'] = pd.to_datetime(df['date'])
df['Hour'] = df['Datetime'].dt.hour
df['DayOfWeek'] = df['Datetime'].dt.dayofweek
df['Day'] = df['Datetime'].dt.day
df['Month'] = df['Datetime'].dt.month

# Drop unnecessary columns
df.drop(['date', 'Datetime'], axis=1, inplace=True)

# Define features and target
features = ['co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3', 'Hour', 'DayOfWeek', 'Day', 'Month']
target = 'co'

X = df[features].values
y = df[target].values

# Normalize features
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Split the data into 70% and 30%
split_index = int(0.7 * len(X))
X_train_test, X_predict = X[:split_index], X[split_index:]
y_train_test, y_predict = y[:split_index], y[split_index:]

# Further split the 70% data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_test, y_train_test, test_size=0.2, random_state=42)

# Reshape data for LNN input
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
X_predict = X_predict.reshape((X_predict.shape[0], 1, X_predict.shape[1]))

# Build the LNN model
wiring = AutoNCP(32, output_size=1)  # 32 hidden units, output size 1
ltc_cell = LTCCell(wiring)

model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(1, X_train.shape[2])),  # Input shape (time steps, features)
    tf.keras.layers.RNN(ltc_cell, return_sequences=False),  # LTC layer
    tf.keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate on test set (part of the 70%)
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE (70% data): {test_mae:.4f}')

# Predict and evaluate on the remaining 30%
y_pred = model.predict(X_predict)
mae_co = np.mean(np.abs(y_predict - y_pred))
print(f'MAE on 30% prediction data: {mae_co:.4f}')


Epoch 1/100
263/263 [==============================] - 14s 16ms/step - loss: 16845968.0000 - mae: 2960.8865 - val_loss: 18555832.0000 - val_mae: 3050.3481
Epoch 2/100
263/263 [==============================] - 5s 19ms/step - loss: 16824926.0000 - mae: 2957.3455 - val_loss: 18527064.0000 - val_mae: 3045.6289
Epoch 3/100
263/263 [==============================] - 3s 10ms/step - loss: 16786788.0000 - mae: 2950.8521 - val_loss: 18479322.0000 - val_mae: 3037.7815
Epoch 4/100
263/263 [==============================] - 2s 8ms/step - loss: 16734531.0000 - mae: 2942.0129 - val_loss: 18418348.0000 - val_mae: 3027.7273
Epoch 5/100
263/263 [==============================] - 3s 12ms/step - loss: 16667517.0000 - mae: 2930.5491 - val_loss: 18340604.0000 - val_mae: 3014.8623
Epoch 6/100
263/263 [==============================] - 3s 11ms/step - loss: 16583101.0000 - mae: 2916.1726 - val_loss: 18243860.0000 - val_mae: 2998.7744
Epoch 7/100
263/263 [==============================] - 2s 8ms/step - loss: 1

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from ncps.wirings import AutoNCP
from ncps.tf import LTCCell

# Load dataset
df = pd.read_csv('delhi_aqi.csv')  # Replace with the actual file name

# Convert 'date' column to datetime and extract useful features
df['Datetime'] = pd.to_datetime(df['date'])
df['Hour'] = df['Datetime'].dt.hour
df['DayOfWeek'] = df['Datetime'].dt.dayofweek
df['Day'] = df['Datetime'].dt.day
df['Month'] = df['Datetime'].dt.month

# Drop unnecessary columns
df.drop(['date', 'Datetime'], axis=1, inplace=True)

# Define features and target
features = ['co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3', 'Hour', 'DayOfWeek', 'Day', 'Month']
target = 'no'

X = df[features].values
y = df[target].values

# Normalize features
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Split the data into 70% and 30%
split_index = int(0.7 * len(X))
X_train_test, X_predict = X[:split_index], X[split_index:]
y_train_test, y_predict = y[:split_index], y[split_index:]

# Further split the 70% data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_test, y_train_test, test_size=0.2, random_state=42)

# Reshape data for LNN input
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
X_predict = X_predict.reshape((X_predict.shape[0], 1, X_predict.shape[1]))

# Build the LNN model
wiring = AutoNCP(32, output_size=1)  # 32 hidden units, output size 1
ltc_cell = LTCCell(wiring)

model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(1, X_train.shape[2])),  # Input shape (time steps, features)
    tf.keras.layers.RNN(ltc_cell, return_sequences=False),  # LTC layer
    tf.keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate on test set (part of the 70%)
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE (70% data): {test_mae:.4f}')

# Predict and evaluate on the remaining 30%
y_pred = model.predict(X_predict)
mae_no = np.mean(np.abs(y_predict - y_pred))
print(f'MAE on 30% prediction data: {mae_no:.4f}')


Epoch 1/50
263/263 [==============================] - 13s 12ms/step - loss: 4793.6875 - mae: 32.7361 - val_loss: 5388.3979 - val_mae: 34.6432
Epoch 2/50
263/263 [==============================] - 2s 8ms/step - loss: 4606.8140 - mae: 32.2198 - val_loss: 5165.7114 - val_mae: 34.5524
Epoch 3/50
263/263 [==============================] - 2s 8ms/step - loss: 4374.6162 - mae: 32.5539 - val_loss: 4893.4019 - val_mae: 35.2708
Epoch 4/50
263/263 [==============================] - 2s 8ms/step - loss: 4134.2622 - mae: 33.6885 - val_loss: 4639.1338 - val_mae: 36.8253
Epoch 5/50
263/263 [==============================] - 3s 11ms/step - loss: 3916.0696 - mae: 35.5459 - val_loss: 4389.6807 - val_mae: 38.6498
Epoch 6/50
263/263 [==============================] - 2s 8ms/step - loss: 3555.4705 - mae: 32.2563 - val_loss: 3658.2778 - val_mae: 23.7976
Epoch 7/50
263/263 [==============================] - 2s 8ms/step - loss: 2844.3450 - mae: 20.0264 - val_loss: 3061.2251 - val_mae: 20.6429
Epoch 8/50
263/26

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from ncps.wirings import AutoNCP
from ncps.tf import LTCCell

# Load dataset
df = pd.read_csv('delhi_aqi.csv')  # Replace with the actual file name

# Convert 'date' column to datetime and extract useful features
df['Datetime'] = pd.to_datetime(df['date'])
df['Hour'] = df['Datetime'].dt.hour
df['DayOfWeek'] = df['Datetime'].dt.dayofweek
df['Day'] = df['Datetime'].dt.day
df['Month'] = df['Datetime'].dt.month

# Drop unnecessary columns
df.drop(['date', 'Datetime'], axis=1, inplace=True)

# Define features and target
features = ['co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3', 'Hour', 'DayOfWeek', 'Day', 'Month']
target = 'no2'

X = df[features].values
y = df[target].values

# Normalize features
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Split the data into 70% and 30%
split_index = int(0.7 * len(X))
X_train_test, X_predict = X[:split_index], X[split_index:]
y_train_test, y_predict = y[:split_index], y[split_index:]

# Further split the 70% data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_test, y_train_test, test_size=0.2, random_state=42)

# Reshape data for LNN input
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
X_predict = X_predict.reshape((X_predict.shape[0], 1, X_predict.shape[1]))

# Build the LNN model
wiring = AutoNCP(32, output_size=1)  # 32 hidden units, output size 1
ltc_cell = LTCCell(wiring)

model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(1, X_train.shape[2])),  # Input shape (time steps, features)
    tf.keras.layers.RNN(ltc_cell, return_sequences=False),  # LTC layer
    tf.keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate on test set (part of the 70%)
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE (70% data): {test_mae:.4f}')

# Predict and evaluate on the remaining 30%
y_pred = model.predict(X_predict)
mae_no2 = np.mean(np.abs(y_predict - y_pred))
print(f'MAE on 30% prediction data: {mae_no2:.4f}')


Epoch 1/50
263/263 [==============================] - 12s 10ms/step - loss: 6449.2915 - mae: 64.5687 - val_loss: 6805.3193 - val_mae: 64.5998
Epoch 2/50
263/263 [==============================] - 2s 9ms/step - loss: 6007.0322 - mae: 61.0341 - val_loss: 6293.0059 - val_mae: 60.5146
Epoch 3/50
263/263 [==============================] - 3s 10ms/step - loss: 5404.0327 - mae: 55.9780 - val_loss: 5566.6753 - val_mae: 54.4931
Epoch 4/50
263/263 [==============================] - 2s 8ms/step - loss: 4592.6064 - mae: 49.0064 - val_loss: 4669.6050 - val_mae: 46.8484
Epoch 5/50
263/263 [==============================] - 2s 8ms/step - loss: 3778.5151 - mae: 41.9864 - val_loss: 3877.4402 - val_mae: 40.2852
Epoch 6/50
263/263 [==============================] - 2s 8ms/step - loss: 3083.6768 - mae: 36.5939 - val_loss: 3213.8547 - val_mae: 36.0083
Epoch 7/50
263/263 [==============================] - 2s 8ms/step - loss: 2591.7937 - mae: 33.7619 - val_loss: 2842.3867 - val_mae: 34.7447
Epoch 8/50
263/26

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from ncps.wirings import AutoNCP
from ncps.tf import LTCCell

# Load dataset
df = pd.read_csv('delhi_aqi.csv')  # Replace with the actual file name

# Convert 'date' column to datetime and extract useful features
df['Datetime'] = pd.to_datetime(df['date'])
df['Hour'] = df['Datetime'].dt.hour
df['DayOfWeek'] = df['Datetime'].dt.dayofweek
df['Day'] = df['Datetime'].dt.day
df['Month'] = df['Datetime'].dt.month

# Drop unnecessary columns
df.drop(['date', 'Datetime'], axis=1, inplace=True)

# Define features and target
features = ['co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3', 'Hour', 'DayOfWeek', 'Day', 'Month']
target = 'o3'

X = df[features].values
y = df[target].values

# Normalize features
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Split the data into 70% and 30%
split_index = int(0.7 * len(X))
X_train_test, X_predict = X[:split_index], X[split_index:]
y_train_test, y_predict = y[:split_index], y[split_index:]

# Further split the 70% data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_test, y_train_test, test_size=0.2, random_state=42)

# Reshape data for LNN input
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
X_predict = X_predict.reshape((X_predict.shape[0], 1, X_predict.shape[1]))

# Build the LNN model
wiring = AutoNCP(32, output_size=1)  # 32 hidden units, output size 1
ltc_cell = LTCCell(wiring)

model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(1, X_train.shape[2])),  # Input shape (time steps, features)
    tf.keras.layers.RNN(ltc_cell, return_sequences=False),  # LTC layer
    tf.keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate on test set (part of the 70%)
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE (70% data): {test_mae:.4f}')

# Predict and evaluate on the remaining 30%
y_pred = model.predict(X_predict)
mae_o3 = np.mean(np.abs(y_predict - y_pred))
print(f'MAE on 30% prediction data: {mae_o3:.4f}')


Epoch 1/50
263/263 [==============================] - 18s 16ms/step - loss: 11484.6387 - mae: 64.8710 - val_loss: 11795.0869 - val_mae: 66.1963
Epoch 2/50
263/263 [==============================] - 3s 12ms/step - loss: 11234.5244 - mae: 64.0203 - val_loss: 11454.2725 - val_mae: 65.2105
Epoch 3/50
263/263 [==============================] - 2s 8ms/step - loss: 10779.2500 - mae: 62.7793 - val_loss: 10798.4492 - val_mae: 63.6817
Epoch 4/50
263/263 [==============================] - 2s 9ms/step - loss: 10024.1406 - mae: 61.1916 - val_loss: 9986.8594 - val_mae: 62.2082
Epoch 5/50
263/263 [==============================] - 3s 11ms/step - loss: 9226.7793 - mae: 60.0593 - val_loss: 9168.0225 - val_mae: 61.2831
Epoch 6/50
263/263 [==============================] - 3s 12ms/step - loss: 8493.4316 - mae: 59.6248 - val_loss: 8470.1426 - val_mae: 61.2785
Epoch 7/50
263/263 [==============================] - 3s 10ms/step - loss: 7734.4741 - mae: 57.2009 - val_loss: 7508.8369 - val_mae: 55.6557
Epoch 8

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from ncps.wirings import AutoNCP
from ncps.tf import LTCCell

# Load dataset
df = pd.read_csv('delhi_aqi.csv')  # Replace with the actual file name

# Convert 'date' column to datetime and extract useful features
df['Datetime'] = pd.to_datetime(df['date'])
df['Hour'] = df['Datetime'].dt.hour
df['DayOfWeek'] = df['Datetime'].dt.dayofweek
df['Day'] = df['Datetime'].dt.day
df['Month'] = df['Datetime'].dt.month

# Drop unnecessary columns
df.drop(['date', 'Datetime'], axis=1, inplace=True)

# Define features and target
features = ['co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3', 'Hour', 'DayOfWeek', 'Day', 'Month']
target = 'so2'

X = df[features].values
y = df[target].values

# Normalize features
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Split the data into 70% and 30%
split_index = int(0.7 * len(X))
X_train_test, X_predict = X[:split_index], X[split_index:]
y_train_test, y_predict = y[:split_index], y[split_index:]

# Further split the 70% data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_test, y_train_test, test_size=0.2, random_state=42)

# Reshape data for LNN input
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
X_predict = X_predict.reshape((X_predict.shape[0], 1, X_predict.shape[1]))

# Build the LNN model
wiring = AutoNCP(32, output_size=1)  # 32 hidden units, output size 1
ltc_cell = LTCCell(wiring)

model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(1, X_train.shape[2])),  # Input shape (time steps, features)
    tf.keras.layers.RNN(ltc_cell, return_sequences=False),  # LTC layer
    tf.keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate on test set (part of the 70%)
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE (70% data): {test_mae:.4f}')

# Predict and evaluate on the remaining 30%
y_pred = model.predict(X_predict)
mae_so2 = np.mean(np.abs(y_predict - y_pred))
print(f'MAE on 30% prediction data: {mae_so2:.4f}')

Epoch 1/50
263/263 [==============================] - 12s 11ms/step - loss: 6670.1426 - mae: 65.0659 - val_loss: 6822.4121 - val_mae: 65.6361
Epoch 2/50
263/263 [==============================] - 2s 8ms/step - loss: 6179.9619 - mae: 61.1870 - val_loss: 6245.0312 - val_mae: 61.0795
Epoch 3/50
263/263 [==============================] - 2s 8ms/step - loss: 5511.0762 - mae: 55.4295 - val_loss: 5417.5620 - val_mae: 53.9132
Epoch 4/50
263/263 [==============================] - 2s 9ms/step - loss: 4673.8931 - mae: 47.5385 - val_loss: 4549.8149 - val_mae: 45.8877
Epoch 5/50
263/263 [==============================] - 3s 10ms/step - loss: 3884.3210 - mae: 40.0351 - val_loss: 3769.5925 - val_mae: 39.0890
Epoch 6/50
263/263 [==============================] - 2s 8ms/step - loss: 3234.6785 - mae: 34.8132 - val_loss: 3171.2273 - val_mae: 34.9375
Epoch 7/50
263/263 [==============================] - 2s 8ms/step - loss: 2794.5435 - mae: 32.4128 - val_loss: 2801.9058 - val_mae: 33.4985
Epoch 8/50
263/26

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from ncps.wirings import AutoNCP
from ncps.tf import LTCCell

# Load dataset
df = pd.read_csv('delhi_aqi.csv')  # Replace with the actual file name

# Convert 'date' column to datetime and extract useful features
df['Datetime'] = pd.to_datetime(df['date'])
df['Hour'] = df['Datetime'].dt.hour
df['DayOfWeek'] = df['Datetime'].dt.dayofweek
df['Day'] = df['Datetime'].dt.day
df['Month'] = df['Datetime'].dt.month

# Drop unnecessary columns
df.drop(['date', 'Datetime'], axis=1, inplace=True)

# Define features and target
features = ['co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3', 'Hour', 'DayOfWeek', 'Day', 'Month']
target = 'pm2_5'

X = df[features].values
y = df[target].values

# Normalize features
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Split the data into 70% and 30%
split_index = int(0.7 * len(X))
X_train_test, X_predict = X[:split_index], X[split_index:]
y_train_test, y_predict = y[:split_index], y[split_index:]

# Further split the 70% data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_test, y_train_test, test_size=0.2, random_state=42)

# Reshape data for LNN input
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
X_predict = X_predict.reshape((X_predict.shape[0], 1, X_predict.shape[1]))

# Build the LNN model
wiring = AutoNCP(32, output_size=1)  # 32 hidden units, output size 1
ltc_cell = LTCCell(wiring)

model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(1, X_train.shape[2])),  # Input shape (time steps, features)
    tf.keras.layers.RNN(ltc_cell, return_sequences=False),  # LTC layer
    tf.keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate on test set (part of the 70%)
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE (70% data): {test_mae:.4f}')

# Predict and evaluate on the remaining 30%
y_pred = model.predict(X_predict)
mae_pm2_5 = np.mean(np.abs(y_predict - y_pred))
print(f'MAE on 30% prediction data: {mae_pm2_5:.4f}')

Epoch 1/50
263/263 [==============================] - 13s 13ms/step - loss: 113524.9297 - mae: 245.3976 - val_loss: 122089.1641 - val_mae: 249.2507
Epoch 2/50
263/263 [==============================] - 2s 8ms/step - loss: 112102.8516 - mae: 242.4680 - val_loss: 120259.8281 - val_mae: 245.5519
Epoch 3/50
263/263 [==============================] - 2s 8ms/step - loss: 109628.1484 - mae: 237.2939 - val_loss: 117014.2578 - val_mae: 238.8531
Epoch 4/50
263/263 [==============================] - 2s 8ms/step - loss: 105890.3438 - mae: 229.2852 - val_loss: 112353.4766 - val_mae: 228.9118
Epoch 5/50
263/263 [==============================] - 3s 12ms/step - loss: 100569.1719 - mae: 217.6011 - val_loss: 106519.4688 - val_mae: 216.2469
Epoch 6/50
263/263 [==============================] - 2s 8ms/step - loss: 94641.8125 - mae: 204.3873 - val_loss: 100175.1641 - val_mae: 202.7583
Epoch 7/50
263/263 [==============================] - 2s 8ms/step - loss: 88286.5391 - mae: 191.0212 - val_loss: 93522.289

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from ncps.wirings import AutoNCP
from ncps.tf import LTCCell

# Load dataset
df = pd.read_csv('delhi_aqi.csv')  # Replace with the actual file name

# Convert 'date' column to datetime and extract useful features
df['Datetime'] = pd.to_datetime(df['date'])
df['Hour'] = df['Datetime'].dt.hour
df['DayOfWeek'] = df['Datetime'].dt.dayofweek
df['Day'] = df['Datetime'].dt.day
df['Month'] = df['Datetime'].dt.month

# Drop unnecessary columns
df.drop(['date', 'Datetime'], axis=1, inplace=True)

# Define features and target
features = ['co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3', 'Hour', 'DayOfWeek', 'Day', 'Month']
target = 'pm10'

X = df[features].values
y = df[target].values

# Normalize features
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Split the data into 70% and 30%
split_index = int(0.7 * len(X))
X_train_test, X_predict = X[:split_index], X[split_index:]
y_train_test, y_predict = y[:split_index], y[split_index:]

# Further split the 70% data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_test, y_train_test, test_size=0.2, random_state=42)

# Reshape data for LNN input
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
X_predict = X_predict.reshape((X_predict.shape[0], 1, X_predict.shape[1]))

# Build the LNN model
wiring = AutoNCP(32, output_size=1)  # 32 hidden units, output size 1
ltc_cell = LTCCell(wiring)

model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(1, X_train.shape[2])),  # Input shape (time steps, features)
    tf.keras.layers.RNN(ltc_cell, return_sequences=False),  # LTC layer
    tf.keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate on test set (part of the 70%)
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE (70% data): {test_mae:.4f}')

# Predict and evaluate on the remaining 30%
y_pred = model.predict(X_predict)
mae_pm10 = np.mean(np.abs(y_predict - y_pred))
print(f'MAE on 30% prediction data: {mae_pm10:.4f}')

Epoch 1/50
263/263 [==============================] - 12s 12ms/step - loss: 168009.1406 - mae: 308.5298 - val_loss: 181375.7031 - val_mae: 314.0916
Epoch 2/50
263/263 [==============================] - 3s 10ms/step - loss: 166296.9219 - mae: 305.7374 - val_loss: 179184.2812 - val_mae: 310.5831
Epoch 3/50
263/263 [==============================] - 2s 8ms/step - loss: 163379.1406 - mae: 300.9394 - val_loss: 175378.0781 - val_mae: 304.3942
Epoch 4/50
263/263 [==============================] - 2s 8ms/step - loss: 158934.3125 - mae: 293.5067 - val_loss: 169864.6094 - val_mae: 295.1989
Epoch 5/50
263/263 [==============================] - 2s 8ms/step - loss: 152628.7656 - mae: 282.5595 - val_loss: 162758.6094 - val_mae: 282.9694
Epoch 6/50
263/263 [==============================] - 2s 9ms/step - loss: 144951.5781 - mae: 268.7771 - val_loss: 153726.5469 - val_mae: 267.1524
Epoch 7/50
263/263 [==============================] - 3s 11ms/step - loss: 135463.3594 - mae: 251.6077 - val_loss: 143784

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from ncps.wirings import AutoNCP
from ncps.tf import LTCCell

# Load dataset
df = pd.read_csv('delhi_aqi.csv')  # Replace with the actual file name

# Convert 'date' column to datetime and extract useful features
df['Datetime'] = pd.to_datetime(df['date'])
df['Hour'] = df['Datetime'].dt.hour
df['DayOfWeek'] = df['Datetime'].dt.dayofweek
df['Day'] = df['Datetime'].dt.day
df['Month'] = df['Datetime'].dt.month

# Drop unnecessary columns
df.drop(['date', 'Datetime'], axis=1, inplace=True)

# Define features and target
features = ['co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3', 'Hour', 'DayOfWeek', 'Day', 'Month']
target = 'nh3'

X = df[features].values
y = df[target].values

# Normalize features
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Split the data into 70% and 30%
split_index = int(0.7 * len(X))
X_train_test, X_predict = X[:split_index], X[split_index:]
y_train_test, y_predict = y[:split_index], y[split_index:]

# Further split the 70% data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_test, y_train_test, test_size=0.2, random_state=42)

# Reshape data for LNN input
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
X_predict = X_predict.reshape((X_predict.shape[0], 1, X_predict.shape[1]))

# Build the LNN model
wiring = AutoNCP(32, output_size=1)  # 32 hidden units, output size 1
ltc_cell = LTCCell(wiring)

model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(1, X_train.shape[2])),  # Input shape (time steps, features)
    tf.keras.layers.RNN(ltc_cell, return_sequences=False),  # LTC layer
    tf.keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate on test set (part of the 70%)
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE (70% data): {test_mae:.4f}')

# Predict and evaluate on the remaining 30%
y_pred = model.predict(X_predict)
mae_nh3 = np.mean(np.abs(y_predict - y_pred))
print(f'MAE on 30% prediction data: {mae_nh3:.4f}')

Epoch 1/50
263/263 [==============================] - 12s 11ms/step - loss: 1281.1456 - mae: 24.9784 - val_loss: 1299.6122 - val_mae: 25.0355
Epoch 2/50
263/263 [==============================] - 2s 9ms/step - loss: 1189.2351 - mae: 23.0787 - val_loss: 1161.3015 - val_mae: 22.1479
Epoch 3/50
263/263 [==============================] - 3s 10ms/step - loss: 1020.3853 - mae: 19.3773 - val_loss: 965.5653 - val_mae: 18.2215
Epoch 4/50
263/263 [==============================] - 2s 8ms/step - loss: 844.9330 - mae: 16.2835 - val_loss: 803.9718 - val_mae: 16.0999
Epoch 5/50
263/263 [==============================] - 2s 8ms/step - loss: 724.5480 - mae: 15.1474 - val_loss: 713.1530 - val_mae: 15.6950
Epoch 6/50
263/263 [==============================] - 2s 8ms/step - loss: 672.4422 - mae: 15.3257 - val_loss: 681.9584 - val_mae: 16.0917
Epoch 7/50
263/263 [==============================] - 2s 8ms/step - loss: 659.1028 - mae: 15.7351 - val_loss: 675.0172 - val_mae: 16.4434
Epoch 8/50
263/263 [======

In [ ]:
avg_mae = ( mae_co + mae_nh3 + mae_pm10 + mae_pm2_5 + mae_so2 + mae_o3 + mae_no + mae_no2 ) / 8
avg_mae

10.2087875